In [1]:
from distutils.command.config import config
%load_ext autoreload
%autoreload 2
import sys
import os
print(os.getcwd())
os.chdir('../../complete_project/../')
print(os.getcwd())
# Then set up the paths
# import sys

# import os
# os.environ['PYTHONPATH'] = os.getcwd()  # Now points to thesis_code directory
# sys.path.append(os.environ['PYTHONPATH'])
sys.path.append("/home/caspar/thesis_code/CellOracle")
sys.path.append("/home/caspar/thesis_code/complete_project/py files")
sys.path.append("/home/caspar/thesis_code/complete_project/py files/AIFiles")
sys.path.append("/home/caspar/thesis_code/complete_project/py files/baseGRNConstructionFiles")
sys.path.append("/home/caspar/thesis_code/complete_project/py files/oracleInferenceFiles")
sys.path.append("/home/caspar/thesis_code/complete_project/py files/oracleSetup")
import scanpy as sc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import anndata
import sys
import logging
from datetime import datetime
import wandb
import optuna
#import modified_celloracle as mco
from CellOracle import celloracle as co

import CellOracleSetup as setup_module
import GRNClusterAnalysis as analysis_module
import GRNInference as inference_module
import GRNInferenceTest as inference_test_module

log_dir = 'logs'
os.makedirs(log_dir, exist_ok=True)
log_filename = os.path.join(log_dir, f"app_{datetime.now().strftime('%Y_%m_%d')}.log")

# Configure the basic logging
logging.basicConfig(
    filename=log_filename,
    filemode='a',
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

/home/caspar/thesis_code/complete_project/notebooks
/home/caspar/thesis_code


/home/caspar/miniconda3/envs/train_model_gpu/lib/python3.10/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/home/caspar/miniconda3/envs/train_model_gpu/lib/python3.10/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This m

ModuleNotFoundError: No module named 'velocyto'

In [2]:
!nvidia-smi


Fri Jun  6 13:09:31 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 555.42.06              Driver Version: 555.42.06      CUDA Version: 12.5     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA RTX 6000 Ada Gene...    Off |   00000000:AC:00.0  On |                  Off |
| 30%   38C    P8             25W /  300W |   14810MiB /  49140MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [8]:

import os
import torch.nn as nn
import AI as ai_module # Import the refactored script
import ParameterTuning as tuning_module # Import the refactored script

data_path_new_data = os.path.join('../celloracle_data', "celloracle_object/new_promoter_without_mescs_trimmed_test_own_umap")
base_config = {
    # Paths
    "ORACLE_PATH": os.path.join(data_path_new_data, "ready_oracle.pkl"),
    "MODEL_SAVE_PATH_BASE": os.path.join("../celloracle_data", "optuna_hpo_models2"), # Base for HPO trial models
    "LOG_DIR_BASE": os.path.join('../celloracle_data', "optuna_hpo_logs2"),         # Base for HPO trial logs
    "WAND_B_LOG_DIR_BASE": os.path.join('../celloracle_data', "optuna_hpo_wandb_logs2"), # Base for HPO trial wandb logs
    "HPO_FIXED_MAX_STEPS_PER_EPISODE": 20, 
    "TRANSITION_MATRIX": os.path.join("../celloracle_data/transition_matrix", "transition_matrix.pkl"), # Path to transition matrix


    "BATCH_SIZE": 256,
    "USE_MASKABLE_PPO": False, 
    "VERBOSE": 0,              
    "LOG_INTERVAL": 10,
    "RESET_NUM_TIMESTEPS": False,
    "DEVICE": "auto",
    "USE_PROGRESS_BAR": False,  
    "STEP_SAVE_FREQ": 1000000000, 
    "PPO_N_STEPS": 512,
    "PPO_N_EPOCHS": 10,
    "GAMMA": 0.99,
    "GAE_LAMBDA": 0.95,
    "CLIP_RANGE": 0.2,
    "ENT_COEF": 0.01,       
    "VF_COEF": 0.5,
    "MAX_GRAD_NORM": 0.5,
    "LEARNING_RATE": 3e-4,  

    # Environment Specific 
    "ALLOW_GENE_ACTIVATION": True,
    "STANDARD_SD_FACTOR": 1.5,
    "USE_LEARNING_RATE_SCHEDULE": False, 
    "USE_SIMILARITY_REWARD": True,
    "ENV_TYPE": "FAST",  #options: FIRST,NEW_INPUT, GENERAL,FAST
    "USE_PREV_KNOCKOUT": True, 

    # ENV HYPERPARAMETERS (Some will be tuned by Optuna)
    "DIVIDER_OF_TOTAL_STEPS_FOR_CURRICULUM_LEARNING": 20,
    "TARGET_CELLS_PER_PHASE": 14,
    "MAX_STEP_INCREASE_PER_PHASE": 8,
    "MAX_STEPS_FIRST_PHASE": 50000, 
    "PHASE_STEP_INCREASE": 15000,
    "MAX_STEPS_PER_EPISODE": 42,
    "OPTUNA_REPORT_INTERVAL": 10000,

    # Other ENV params
    "GENE_ACTIVITY_THRESHOLD": 0.01,
    "TARGET_DISTANCE_THRESHOLD": 0.1,
    "DISTANCE_GAMMA": 0.99,           
    "STEP_PENALTY": -0.05,
    "GOAL_BONUS": 10,
    "FAIL_PENALTY": -3,
    "SAME_CELL_PENALTY": -0.5,
    "DISTANCE_REWARD_SCALE": 5,
    "PARAMETER_TUNING":True,
    "TOTAL_TIMESTEPS_HPO": 500000,
    "STUDY_NAME": "ppo_celloracle_hpo_general40",
    "STORAGE_NAME": "sqlite:///ppo_celloracle_hpo_general40.db",

    # NN Architecture (Fixed for this example, but can be tuned)
    "PI_ARCH": [512,256],
    "VF_ARCH": [512,256],
    "ACTIVATION_FN": nn.ReLU, # Make sure nn is imported if you use this directly
    
}

if base_config["USE_SIMILARITY_REWARD"]:
    base_config["DISTANCE_REWARD_SCALE"] = 50
else:
    base_config["DISTANCE_REWARD_SCALE"] = 5

study_name = base_config["STUDY_NAME"]

trained_model = tuning_module.run_optuna_hpo(base_config)


[I 2025-09-09 14:34:39,454] Using an existing study with name 'ppo_celloracle_hpo_general40' instead of creating a new one.


running number of trials: 4

--- Starting Optuna HPO Study: ppo_celloracle_hpo_general40 ---
test

---> Starting Optuna Trial 1 <---
  --- Hyperparameters for this Trial ---
  PPO Agent:
    Learning Rate: 0.000071
    N_Steps: 256
    N_Epochs: 7
    Clip Range: 0.1468
    Entropy Coef: 0.001162
    Gamma (Discount): 0.9946
  Reward Function:
    Goal Bonus: 1.1853
    Fail Penalty: -3.6273
    Step Penalty: -0.0918
    Same Cell Penalty: -0.5903
    Distance Reward Scale: 67.9896
    Distance Gamma (PBRS): 0.9685
  ------------------------------------


--- Starting Training Run ---
Configuration: {'ORACLE_PATH': '../celloracle_data/celloracle_object/new_promoter_without_mescs_trimmed_test_own_umap/ready_oracle.pkl', 'MODEL_SAVE_PATH_BASE': '../celloracle_data/optuna_hpo_models2', 'LOG_DIR_BASE': '../celloracle_data/optuna_hpo_logs2', 'WAND_B_LOG_DIR_BASE': '../celloracle_data/optuna_hpo_wandb_logs2', 'HPO_FIXED_MAX_STEPS_PER_EPISODE': 20, 'TRANSITION_MATRIX': '../celloracle_data/transition_matrix/transition_matrix.pkl', 'BATCH_SIZE': 256, 'USE_MASKABLE_PPO': False, 'VERBOSE': 0, 'LOG_INTERVAL': 10, 'RESET_NUM_TIMESTEPS': False, 'DEVICE': 'auto', 'USE_PROGRESS_BAR': False, 'STEP_SAVE_FREQ': 1000000000, 'PPO_N_STEPS': 256, 'PPO_N_EPOCHS': 7, 'GAMMA': 0.9946470458309974, 'GAE_LAMBDA': 0.95, 'CLIP_RANGE': 0.1467983561008608, 'ENT_COEF': 0.0011616722433639892, 'VF_COEF': 0.5, 'MAX_GRAD_NORM': 0.5, 'LEARNING_RATE': 7.106591851092232e-05, 'ALLOW_GENE_ACTIVATION': True, 'STANDARD_SD_FACTOR': 1.5, 'USE_LEARNING_RATE_SCHEDULE': False, 'USE_SIM

PPO_update/approx_kl,▅▅▅▅▅▅▁▁▁▁▁▁▄▄▄▄▄▄▄▄▃▃▄▄▄▄██████████████
PPO_update/clip_fraction,▂▂▂▂▂▁▁▁▁▁▃▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▅▅▅▅▇▇▇▇▇█████
PPO_update/entropy_loss,▁▁▁▁▁▂▂▂▂▂▂▂▂▂▂▂▂▂▂▃▃▃▃▃▃▅▅▅▅▅▆▆▆▆██████
PPO_update/explained_variance,▁▁▁▁▁▅▅▅▅▅▇▇▇▇▇▇▇███████████████████████
PPO_update/value_loss,▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▄▄▄▄▄▄▄▄▆▆▆████
batch_diversity/num_unique_current_types,▅▃▃▇▁▂▅▁▃▃▅▇▂▁▅▁▇▂▂▁▃▇▁▂▆▆▃▅█▇▅▃▆▃▅▅▇▇▇▄
batch_diversity/num_unique_target_types,▁▁▁███████████▁█▁▁▁███▁▁██████████████▁▁
diagnostics/same_cell_event_count_period,▆█▇▄▅▃▆▃▅▄▇▅▆▄▄▄▆▁▄▁▄▄▂▃▇▂▄▃▆▇▄▅▃▄▃▁▁▂▂▆
episode_overall/success_rate,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
episode_overall/total_episodes_collected,▁▁▁▂▂▂▂▂▂▂▃▃▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▆▆▆▆▆▆▇▇▇▇███
episode_period/ep_len_mean,▁▇▃██▆██▇█▇███▇██▇█████▇██▇██▇▇███▇▇▇███


[I 2025-09-09 14:36:26,052] Trial 1 finished with value: 356.0 and parameters: {'LEARNING_RATE': 7.106591851092232e-05, 'PPO_N_STEPS': 256, 'PPO_N_EPOCHS': 7, 'CLIP_RANGE': 0.1467983561008608, 'ENT_COEF': 0.0011616722433639892, 'GAMMA': 0.9946470458309974, 'STEP_PENALTY': -0.09179929788622242, 'FAIL_PENALTY': -3.627346799835591, 'GOAL_BONUS': 1.185260448662222, 'SAME_CELL_PENALTY': -0.5902704435140169, 'DISTANCE_REWARD_SCALE': 67.9896242159113, 'DISTANCE_GAMMA': 0.968493564427131}. Best is trial 1 with value: 356.0.
[W 2025-09-09 14:36:26,601] Trial 2 failed with parameters: {'LEARNING_RATE': 4.5598044903929383e-05, 'PPO_N_STEPS': 1024, 'PPO_N_EPOCHS': 11, 'CLIP_RANGE': 0.1873687420594126, 'ENT_COEF': 0.01223705789444759} because of the following error: StorageInternalError('An exception is raised during the commit. This typically happens due to invalid data in the commit, e.g. exceeding max length. ').
Traceback (most recent call last):
  File "/home/caspar/miniconda3/envs/gpu_cell_or

ERROR weird: An exception is raised during the commit. This typically happens due to invalid data in the commit, e.g. exceeding max length. 

--- Optuna Study Finished ---
Study dashboard command: optuna-dashboard sqlite:///ppo_celloracle_hpo_general40.db
Number of finished trials in study: 3


Traceback (most recent call last):
  File "/home/caspar/miniconda3/envs/gpu_cell_oracle/lib/python3.10/site-packages/sqlalchemy/engine/base.py", line 1141, in _commit_impl
    self.engine.dialect.do_commit(self.connection)
  File "/home/caspar/miniconda3/envs/gpu_cell_oracle/lib/python3.10/site-packages/sqlalchemy/engine/default.py", line 706, in do_commit
    dbapi_connection.commit()
sqlite3.OperationalError: disk I/O error

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/home/caspar/miniconda3/envs/gpu_cell_oracle/lib/python3.10/site-packages/optuna/storages/_rdb/storage.py", line 77, in _create_scoped_session
    session.commit()
  File "/home/caspar/miniconda3/envs/gpu_cell_oracle/lib/python3.10/site-packages/sqlalchemy/orm/session.py", line 2032, in commit
    trans.commit(_to_root=True)
  File "<string>", line 2, in commit
  File "/home/caspar/miniconda3/envs/gpu_cell_oracle/lib/python3.10/site-packages/sqlalchemy

In [7]:
study_name = "ppo_celloracle_hpo_general30"
study =  optuna.create_study(
            study_name=base_config["STUDY_NAME"],
            storage=base_config["STORAGE_NAME"],
            direction="maximize",
            load_if_exists=True
        )
try:
    pruned_trials = len([t for t in study.trials if t.state == optuna.trial.TrialState.PRUNED])
    complete_trials = len([t for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE])
    failed_trials = len([t for t in study.trials if t.state == optuna.trial.TrialState.FAIL])
    print(f"  Trials: Completed={complete_trials}, Pruned={pruned_trials}, Failed={failed_trials}")

    if complete_trials > 0:
        best_trial = study.best_trial
        print("\nBest trial:")
        print(f"  Value (Maximized Metric): {best_trial.value:.4f}")
        print("  Params: ")
        for key, value in best_trial.params.items():
            print(f"    {key}: {value}")

        # Save best parameters to a file
        best_params_filename = os.path.join(base_config["MODEL_SAVE_PATH_BASE"],
                                            f"{study_name}_best_params.txt")
        with open(best_params_filename, "w") as f:
            f.write(f"Study Name: {study_name}\n")
            f.write(f"Best trial number: {best_trial.number}\n")
            f.write(f"Best trial value: {best_trial.value}\n\n")
            f.write("Best Hyperparameters:\n")
            for key, value in best_trial.params.items():
                f.write(f"  {key}: {value}\n")
        print(f"Best parameters saved to {best_params_filename}")
    else:
        print("No trials completed successfully, cannot determine the best one.")
except ValueError:
    print("ERROR: No completed trials found.")
except Exception as e_results:
    print("WRONG", e_results)


[I 2025-09-09 13:59:19,810] Using an existing study with name 'ppo_celloracle_hpo_general30' instead of creating a new one.


  Trials: Completed=2, Pruned=0, Failed=1

Best trial:
  Value (Maximized Metric): 198.0000
  Params: 
    LEARNING_RATE: 7.106591851092232e-05
    PPO_N_STEPS: 256
    PPO_N_EPOCHS: 7
    CLIP_RANGE: 0.1467983561008608
    ENT_COEF: 0.0011616722433639892
    GAMMA: 0.9946470458309974
    STEP_PENALTY: -0.09179929788622242
    FAIL_PENALTY: -3.627346799835591
    GOAL_BONUS: 1.185260448662222
    SAME_CELL_PENALTY: -0.5902704435140169
    DISTANCE_REWARD_SCALE: 67.9896242159113
    DISTANCE_GAMMA: 0.968493564427131
Best parameters saved to ../celloracle_data/optuna_hpo_models2/ppo_celloracle_hpo_general30_best_params.txt


In [20]:
#lts see if gpus work

import os
import torch.nn as nn
import AI as ai_module # Import the refactored script
import ParameterTuning as tuning_module # Import the refactored script
import multiprocessing
import time
    
def worker_process(gpu_id, base_config):
    os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu_id)
    print(f"Worker {gpu_id} started, using GPU {gpu_id}")
    tuning_module.run_optuna_hpo(base_config,gpu_id=gpu_id)
    
def start():
    multiprocessing.set_start_method('spawn', force=True)
    data_path_new_data = os.path.join('../celloracle_data', "celloracle_object/new_promoter_without_mescs_trimmed_test_own_umap")
    base_config = {
        # Paths
        "ORACLE_PATH": os.path.join(data_path_new_data, "ready_oracle.pkl"),
        "MODEL_SAVE_PATH_BASE": os.path.join("../celloracle_data", "optuna_hpo_models2"), # Base for HPO trial models
        "LOG_DIR_BASE": os.path.join('../celloracle_data', "optuna_hpo_logs2"),         # Base for HPO trial logs
        "WAND_B_LOG_DIR_BASE": os.path.join('../celloracle_data', "optuna_hpo_wandb_logs2"), # Base for HPO trial wandb logs
        "HPO_FIXED_MAX_STEPS_PER_EPISODE": 20, 
        "TRANSITION_MATRIX": os.path.join("../celloracle_data/transition_matrix", "transition_matrix.pkl"), # Path to transition matrix
    
    
        "BATCH_SIZE": 256,
        "USE_MASKABLE_PPO": False, 
        "VERBOSE": 0,              
        "LOG_INTERVAL": 10,
        "RESET_NUM_TIMESTEPS": False,
        "DEVICE": "auto",
        "USE_PROGRESS_BAR": False,  
        "STEP_SAVE_FREQ": 1000000000, 
        "PPO_N_STEPS": 512,
        "PPO_N_EPOCHS": 10,
        "GAMMA": 0.99,
        "GAE_LAMBDA": 0.95,
        "CLIP_RANGE": 0.2,
        "ENT_COEF": 0.01,       
        "VF_COEF": 0.5,
        "MAX_GRAD_NORM": 0.5,
        "LEARNING_RATE": 3e-4,  
    
        # Environment Specific 
        "ALLOW_GENE_ACTIVATION": True,
        "STANDARD_SD_FACTOR": 1.5,
        "USE_LEARNING_RATE_SCHEDULE": False, 
        "USE_SIMILARITY_REWARD": True,
        "ENV_TYPE": "FAST",  #options: FIRST,NEW_INPUT, GENERAL,FAST
        "USE_PREV_KNOCKOUT": False, 
    
        # ENV HYPERPARAMETERS (Some will be tuned by Optuna)
        "DIVIDER_OF_TOTAL_STEPS_FOR_CURRICULUM_LEARNING": 20,
        "TARGET_CELLS_PER_PHASE": 14,
        "MAX_STEP_INCREASE_PER_PHASE": 8,
        "MAX_STEPS_FIRST_PHASE": 50000, 
        "PHASE_STEP_INCREASE": 15000,
        "MAX_STEPS_PER_EPISODE": 42,
        "OPTUNA_REPORT_INTERVAL": 10000,
    
        # Other ENV params
        "GENE_ACTIVITY_THRESHOLD": 0.01,
        "TARGET_DISTANCE_THRESHOLD": 0.1,
        "DISTANCE_GAMMA": 0.99,           
        "STEP_PENALTY": -0.05,
        "GOAL_BONUS": 10,
        "FAIL_PENALTY": -3,
        "SAME_CELL_PENALTY": -0.5,
        "DISTANCE_REWARD_SCALE": 5,
        "PARAMETER_TUNING":True,
        "TOTAL_TIMESTEPS_HPO": 4000000,
        "STUDY_NAME": "ppo_celloracle_hpo_general10",
        "STORAGE_NAME": "sqlite:///ppo_celloracle_hpo_general10.db",
    
        # NN Architecture (Fixed for this example, but can be tuned)
        "PI_ARCH": [512,256],
        "VF_ARCH": [512,256],
        "ACTIVATION_FN": nn.ReLU, # Make sure nn is imported if you use this directly
        
    }
    
    if base_config["USE_SIMILARITY_REWARD"]:
        base_config["DISTANCE_REWARD_SCALE"] = 50
    else:
        base_config["DISTANCE_REWARD_SCALE"] = 5

        
    num_gpus = 2
    processes = []
    for gpu_id in range(num_gpus):
        print("test")
        p = multiprocessing.Process(target=worker_process, args=(gpu_id, base_config))
        processes.append(p)
        p.start()
        time.sleep(5)  # Stagger the start time slightly
        
    for p in processes:
        p.join()
        
if __name__ == '__main__':
    start()


test


Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/home/caspar/miniconda3/envs/gpu_cell_oracle/lib/python3.10/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/home/caspar/miniconda3/envs/gpu_cell_oracle/lib/python3.10/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'worker_process' on <module '__main__' (built-in)>


test


Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/home/caspar/miniconda3/envs/gpu_cell_oracle/lib/python3.10/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/home/caspar/miniconda3/envs/gpu_cell_oracle/lib/python3.10/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'worker_process' on <module '__main__' (built-in)>
